# Making Topic Usage Graph
This python library creates a bar graph of clusters and cluster favorance values for analysis across multiple subreddits.

In [1]:
# Import useful mathematical libraries
import copy
import numpy as np
import pandas as pd

# Import useful Machine learning libraries
import gensim

#Import nltk list of stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

#Import Graph Functions
import matplotlib.pyplot as plt
from brokenaxes import brokenaxes
import matplotlib.gridspec as gridspec
import chart_studio.plotly as py
import chart_studio.tools as tls

#import function for normalization
from sklearn.preprocessing import normalize

#import TF function
from sklearn.feature_extraction.text import CountVectorizer


# Import utility files
from utils import read_df, remove_links, clean_sentence, save_object, load_object, generate_cluster_favorance, make_clustering_objects,make_frequency_dictionary

from pprint import pprint

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/eliseglaser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#Enter Model Name from BuildModelsAndMatricies
model_name = '2019_SW_PandC'
#Enter Subreddit Abbreviation List from BuildModelsAndMatricies
subredditAbr = ['Posts','Comments']
#Enter Subreddit Real Names
subreddits = ['Posts','Responses']
sumNormMatrix = generate_cluster_favorance(model_name,subredditAbr)      

In [7]:
subredditAbr = ['Posts','Responses']
#Enter colors for your corresponding subreddits
colors = [(0.133, 0.584, 0.882),(0.9, 0.066, 0.035),(0.901, 0.635, 0.101),(0.133, 0.882, 0.607)]

#Graph The Results

#initializing variables
w = 0.8/len(subredditAbr)
index = 0
x_Shift = -1*w
x = np.linspace(1,100,100)
#bax = brokenaxes(ylims=((0,0.05), (0.5, 0.6)), hspace=.05)
#fig,(ax1,ax2) = plt.subplots(2,1,sharex=True,figsize=(200,50))
ylimratio = (0.15)
ylim2ratio = (0.85)
gs = gridspec.GridSpec(2, 1, height_ratios=[ylimratio, ylim2ratio])
fig = plt.figure(figsize=(200,100))
ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1])
plt.subplots_adjust(hspace=0.04)

#making bar graphs
for matrix in sumNormMatrix:
    ax1.bar(x+x_Shift, matrix.ravel(), width=w, color=colors[index], align='center',label=subredditAbr[index])
    ax2.bar(x+x_Shift, matrix.ravel(), width=w, color=colors[index], align='center',label=subredditAbr[index])
    x_Shift=x_Shift+w
    index=index+1
    
    
#Adjust Labels and Sizes   
ax2.set_ylabel('% Cluster',fontsize=175)
ax2.yaxis.set_label_coords(0.10, 0.5, transform=fig.transFigure)
#ax2.legend(loc='upper right',prop={'size':100})
plt.legend(bbox_to_anchor=(0.2,0.86),\
    bbox_transform=plt.gcf().transFigure,prop={'size':125})
plt.xticks(x)
ax1.tick_params(axis='both', which='major', labelsize=100)
ax2.tick_params(axis='both', which='major', labelsize=100)


#Breaking the y-axis
ax1.spines['bottom'].set_visible(False)
ax1.tick_params(axis='x',which='both',bottom=False)
ax2.spines['top'].set_visible(False)
ax2.set_ylim(0,0.045)
ax1.set_ylim(0.5,0.6)
ax1.set_yticks(np.arange(0.5,0.61,0.05))
ax1.set_xlim([0.5,101])
ax2.set_xlim([0.5,101])
ax1.tick_params(labeltop=False)
ax1.tick_params(labelbottom=False)
plt.xticks(rotation='vertical')


#Make Break Lines
kwargs = dict(color='k', clip_on=False)
xlim = ax1.get_xlim()
ylim = ax1.get_ylim()
ylim2 = ax2.get_ylim()
dx = .02*(xlim[1]-xlim[0])
dy = .01*(.1/.15)
ax1.plot((xlim[0]-0.5*dx,xlim[0]+0.5*dx), (ylim[0]-0.5*dy,ylim[0]+0.5*dy), **kwargs)
ax1.plot((xlim[1]-0.5*dx,xlim[1]+0.5*dx), (ylim[0]-0.5*dy,ylim[0]+0.5*dy), **kwargs)
dy = .01*(.045)/.85
ax2.plot((xlim[0]-0.5*dx,xlim[0]+0.5*dx), (ylim2[1]-0.5*dy,ylim2[1]+0.5*dy), **kwargs)
ax2.plot((xlim[1]-0.5*dx,xlim[1]+0.5*dx), (ylim2[1]-0.5*dy,ylim2[1]+0.5*dy), **kwargs)

plt.show()

# Make CSV of Top Words
The following code makes a sorted list of clusters and their top words by frequency.
In the utils function you will also find a function that can print out the top n words from k cluser in a nice printed format, or even make a csv for you as well. Useful for presentations and writing the paper.

In [8]:
num_clusters = 100
model = gensim.models.Word2Vec.load('models/' + model_name + '.model')
kmeans = load_object('clusters/', model_name + "-words-cluster_model-" + str(num_clusters))
WordsByFeatures = load_object('matricies/', model_name + '-' + 'WordsByFeatures')
vocab_list = sorted(list(model.wv.vocab))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [21]:
clusters = make_clustering_objects(model, kmeans, vocab_list, WordsByFeatures)

In [22]:
#Sort by word frequency
for i in range(len(clusters)):
    listUnsorted = clusters[i]['word_list']
    vocab_sorted = sorted(listUnsorted, key=lambda x: -x[1])


In [23]:
# Set the number of words to display. The table with contain the top size_words_list words
size_words_list = 15
table = []
for i in range(len(clusters)):
    row = []
    row.append("cluster " + str(i+1))
    row.append(clusters[i]["total_freq"])
    row.append(clusters[i]["unique_words"])
    word_str = ""
    for j in range(size_words_list):
        try:
            row.append(clusters[i]["word_list"][j])
            word_str += " " + str(clusters[i]["word_list"][j])
        except:
            break
    row.append(word_str)
    table.append(row)

In [24]:
#Make Headers
import csv
with open('cluster-analysis/' + model_name + "-" + str(num_clusters) + '.csv', 'w') as csvfile:
    labels = ["Cluster Num", "total_freq", "unique_words"]
    for i in range(size_words_list):
        labels.append("Word" + str(i+1))
    labels.append("word_list")
    writer = csv.writer(csvfile)
    writer.writerow(labels)
    [writer.writerow(r) for r in table]

In [25]:
filepath = 'cluster-analysis/' + model_name + "-" + str(num_clusters) + '.csv'
df = pd.read_csv(filepath)
df.head(1500)

,Cluster Num,total_freq,unique_words,Word1,Word2,Word3,Word4,Word5,Word6,Word7,Word8,Word9,Word10,Word11,Word12,Word13,Word14,Word15,word_list
0,cluster 1,52947,650,"('abs', 61)","('acne', 627)","('acne_acne', 13)","('acne_scars', 45)","('adams_apple', 13)","('adorable', 142)","('aesthetically', 18)","('aesthetics', 23)","('alopecia', 29)","('always_swim_cycle', 23)","('amp_apos', 18)","('androgynous', 29)","('anorexic', 236)","('anyone_colour_pattern', 23)","('appearance', 1222)","('abs', 61) ('acne', 627) ('acne_acne', 13) (..."
1,cluster 2,168323,1269,"('able_afford', 482)","('able_afford_rent', 28)","('able_become_emancipated', 23)","('access_computer', 22)","('access_internet', 65)","('accomodation', 18)","('accrue', 17)","('accrued', 38)","('accruing', 16)","('accumulated', 134)","('accumulating', 74)","('adding_stress', 30)","('afford', 2280)","('afford_apartment', 21)","('afford_food', 33)","('able_afford', 482) ('able_afford_rent', 28)..."
2,cluster 3,179509,802,"('abusive_relationship_seek_help', 23)","('acc', 24)","('account', 2445)","('account_deleted', 23)","('active_member', 13)","('additional_info', 15)","('admins', 48)","('advertise_helper', 11)","('advertising_helper', 24)","('affraid_ask_help', 24)","('allow_fundraising_kind', 29)","('also_check_site', 30)","('alt', 73)","('alt_account', 55)","('alternate_account', 23)","('abusive_relationship_seek_help', 23) ('acc'..."
3,cluster 4,86592,1056,"('able_acknowledge_achievements', 23)","('able_connect_subject', 23)","('able_cover_rent', 26)","('able_live_independently', 27)","('able_persevere_definitely', 23)","('accept_made_mistake', 24)","('accusing_obsessive_thoughts', 23)","('acetaminophen_overdose_horrible', 23)","('actions_impact', 30)","('actually_gaining_muscle', 23)","('actually_googled_question', 23)","('actually_liked_ghost', 23)","('addiction_hands', 23)","('additional_support_hang', 23)","('adjust_economic_world', 23)","('able_acknowledge_achievements', 23) ('able_..."
4,cluster 5,938232,1338,"('abandon', 786)","('able_create_deeper', 23)","('absence', 523)","('absolute', 1631)","('absolutely_fantastic', 32)","('absolution', 19)","('abundance', 72)","('acknowledgment', 20)","('acomplishment', 10)","('act_kindness', 20)","('actions_consequences', 17)","('actively_attempting', 11)","('addition_size_really', 23)","('admiration', 86)","('admire', 513)","('abandon', 786) ('able_create_deeper', 23) (..."
5,cluster 6,1195,47,"('aan', 12)","('al', 197)","('bent', 119)","('bleek', 12)","('daar', 20)","('dan', 75)","('dat', 45)","('dat_er', 10)","('dat_je', 17)","('depressie', 10)","('doen', 16)","('een', 28)","('erg_kut_maar', 10)","('goed', 11)","('goede', 15)","('aan', 12) ('al', 197) ('bent', 119) ('bleek..."
6,cluster 7,104967,1127,"('abuse_suffer_horribly', 23)","('accept_possibly_crowd', 23)","('accept_utterly', 23)","('accept_w', 23)","('accepting_human_beings', 23)","('accomplishment_big_deal', 23)","('actions_little_humpty', 23)","('actions_unless_direct', 23)","('actual_circumstances', 26)","('actually_particular_myth', 23)","('actually_scared_represent', 23)","('admire_resolve_going', 23)","('afford_toxic', 23)","('age_comes_maturity', 23)","('age_unpredictable_times', 23)","('abuse_suffer_horribly', 23) ('accept_possib..."
7,cluster 8,242,6,"('personal_trainer', 75)","('push_ups', 73)","('save_money_transformation', 23)","('spend_form_ur', 23)","('start_cleaning_house', 25)","('transformation_e_g', 23)","('personal_trainer', 75) ('push_ups', 73) ('s...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,cluster 9,117,3,"('empty_hole', 55)","('gaping_hole', 38)","('huge_void', 24)","('empty_hole', 55) ('gaping_hole', 38) ('huge...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,cluster 10,326341,2007,"('abide', 35)","('ability_communicate', 23)","('ability_function', 40)","('abnormal', 344)","('abrupt', 69)","('absolutely_necessary', 50)","('absorb_knowledge', 11)","('absurdly', 52)","('abuseds', 10)","('acceptable', 504)","('acceptance', 772)","('acclimated